## ***Interpolate to 1x1***

In [2]:
from scipy.spatial.distance import cdist
from typing import Iterable

def inverse_distance_weighting(known_points: np.ndarray,
                               unknown_location: Iterable,
                               number_of_neighbours=5,
                               power=2.) -> float:

    # Pre-process unknown location parameter
    if not isinstance(unknown_location, np.ndarray):
        unknown_location = np.array(unknown_location)

    if len(unknown_location.shape) != len(known_points.shape):
        unknown_location = unknown_location[np.newaxis, ...]

    # Calculate distances using scipy.spatial.distance.cdist
    distances = cdist(unknown_location, known_points[:, :-1])

    # Check if any distance is equal to 0 - then return this value
    if not np.all(distances[0]):

        zer_pos = np.where(distances == 0)
        result = known_points[zer_pos[1], -1][0]
        return result

    # Get n closest neighbours...
    sdists = distances.argsort()
    sdists = sdists[0, :number_of_neighbours]
    dists = distances[0, sdists]
    values = known_points[sdists].copy()
    values = values[:, -1]

    # Create weights
    weights = 1 / dists**power

    # Estimate value
    result = np.sum(weights * values) / np.sum(weights)
    return result

## ***Read Pr Data***

In [86]:
import xarray as xr

nc_file = "D:\Min\Review GCM\pr_day_CanESM5_ssp585_r1i1p1f1_gn_20150101-21001231.nc"

ds = xr.open_dataset(nc_file)
 
pr_data = ds['pr']
ds.close()

In [87]:
pr_data = pr_data* 1000 * 60 * 60 * 24

In [10]:
import numpy as np
from scipy.spatial.distance import cdist
from typing import Iterable

def inverse_distance_weighting(known_points: np.ndarray,
                               unknown_location: Iterable,
                               number_of_neighbours=5,
                               power=2.) -> float:

    # Pre-process unknown location parameter
    if not isinstance(unknown_location, np.ndarray):
        unknown_location = np.array(unknown_location)

    if len(unknown_location.shape) != len(known_points.shape):
        unknown_location = unknown_location[np.newaxis, ...]

    # Calculate distances using scipy.spatial.distance.cdist
    distances = cdist(unknown_location, known_points[:, :-1])

    # Check if any distance is equal to 0 - then return this value
    if not np.all(distances[0]):

        zer_pos = np.where(distances == 0)
        result = known_points[zer_pos[1], -1][0]
        return result

    # Get n closest neighbours...
    sdists = distances.argsort()
    sdists = sdists[0, :number_of_neighbours]
    dists = distances[0, sdists]
    values = known_points[sdists][:, -1]

    # Create weights
    weights = 1 / dists**power

    # Estimate value using vectorized operations
    result = np.sum(weights * values) / np.sum(weights)
    return result


In [42]:
import xarray as xr

nc_file = "tas_day_CanESM5_ssp585_r1i1p1f1_gn_20150101-21001231.nc"

ds = xr.open_dataset(nc_file)
 
tas_data = ds['tas']
ds.close()

In [45]:
tas_data = tas_data - 273.15

In [5]:
file_pattern = "D:\Min\Review GCM\hur\*.nc"

combined_dataset = xr.open_mfdataset(file_pattern, combine='by_coords')

In [6]:
hur_data = combined_dataset["hur"].sel(plev=100000)

In [29]:
import numpy as np

def calculate_dewpoint(temp, humidity):
    A = 17.27
    B = 237.7
    alpha = ((A * temp) / (B + temp)) + np.log(humidity/100.0)
    return (B * alpha) / (A - alpha)


In [79]:
tdew = xr.apply_ufunc(
    calculate_dewpoint, 
    tas_data, 
    hur_data,
    input_core_dims=[['time', 'lat', 'lon'], ['time', 'lat', 'lon']],
    output_core_dims=[['time', 'lat', 'lon']],
    vectorize=True,
    dask='parallelized'
)

In [106]:
tdew = tdew.rename('tdew')

In [108]:
merged_dataset = xr.merge([pr_data.rename('pr'), tdew.rename('tdew')])

In [109]:
merged_dataset

<xarray.Dataset>
Dimensions:  (time: 31390, lat: 64, lon: 128)
Coordinates:
  * time     (time) object 2015-01-01 12:00:00 ... 2100-12-31 12:00:00
  * lat      (lat) float64 -87.86 -85.1 -82.31 -79.53 ... 79.53 82.31 85.1 87.86
  * lon      (lon) float64 0.0 2.812 5.625 8.438 ... 348.8 351.6 354.4 357.2
    height   float64 ...
    plev     float64 1e+05
Data variables:
    pr       (time, lat, lon) float32 0.0 0.0 0.0 ... 1.516e+03 1.485e+03
    tdew     (time, lat, lon) float32 dask.array<chunksize=(31390, 64, 128), meta=np.ndarray>

In [117]:
merged_dataset

<xarray.Dataset>
Dimensions:  (time: 31390, lat: 64, lon: 128)
Coordinates:
  * time     (time) object 2015-01-01 12:00:00 ... 2100-12-31 12:00:00
  * lat      (lat) float64 -87.86 -85.1 -82.31 -79.53 ... 79.53 82.31 85.1 87.86
  * lon      (lon) float64 0.0 2.812 5.625 8.438 ... 348.8 351.6 354.4 357.2
    height   float64 ...
    plev     float64 1e+05
Data variables:
    pr       (time, lat, lon) float32 0.0 0.0 0.0 ... 1.516e+03 1.485e+03
    tdew     (time, lat, lon) float32 dask.array<chunksize=(31390, 64, 128), meta=np.ndarray>

In [121]:
rain_days = merged_dataset.where(merged_dataset['pr'] >= 0.1, drop=True)

In [122]:
rain_days

<xarray.Dataset>
Dimensions:  (time: 31390, lat: 64, lon: 128)
Coordinates:
  * time     (time) object 2015-01-01 12:00:00 ... 2100-12-31 12:00:00
  * lat      (lat) float64 -87.86 -85.1 -82.31 -79.53 ... 79.53 82.31 85.1 87.86
  * lon      (lon) float64 0.0 2.812 5.625 8.438 ... 348.8 351.6 354.4 357.2
    height   float64 2.0
    plev     float64 1e+05
Data variables:
    pr       (time, lat, lon) float32 nan nan nan ... 1.516e+03 1.485e+03
    tdew     (time, lat, lon) float32 dask.array<chunksize=(31390, 64, 128), meta=np.ndarray>